In [3]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-u42sivmv/unsloth_c142ee0eb8014230a30cd36899577625
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-u42sivmv/unsloth_c142ee0eb8014230a30cd36899577625
  Resolved https://github.com/unslothai/unsloth.git to commit ec1757c1a02175851146ff5f6ab2a26c8c863fc8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 147.8 MB/s eta 0:00:00

In [4]:
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import load_dataset
from openai import OpenAI

# Configuration
max_seq_length = 2048 # Adjust based on your review lengths
dtype = None
load_in_4bit = True # Use 4-bit quantization to save memory

# Load the Typhoon Model
# using a common alias or the likely path: 'scb10x/typhoon-v2-7b-instruct' or similar
model_name = "scb10x/typhoon-v1.5-8b-instruct"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# Load and Format Your Data
dataset = load_dataset("json", data_files="training_data.jsonl", split="train")

def formatting_prompts_func(examples):
    convos = examples["messages"]
    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]
    return { "text" : texts }

dataset = dataset.map(formatting_prompts_func, batched = True,)

# Add LoRA Adapters (Fine-tuning configuration)
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Rank
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

# Initialize Trainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can set to True for speed boost
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60, # Adjust based on dataset size
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

# Train the model
trainer_stats = trainer.train()

# Save the model
model.save_pretrained("typhoon_fake_review_detector")
tokenizer.save_pretrained("typhoon_fake_review_detector")

print("Fine-tuning complete. Adapters saved to 'typhoon_fake_review_detector'.")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.1.2: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/449 [00:00<?, ?B/s]

scb10x/typhoon-v1.5-8b-instruct does not have a padding token! Will use pad_token = <|reserved_special_token_250|>.


FileNotFoundError: Unable to find '/content/training_data.jsonl'

In [ ]:
from unsloth import FastLanguageModel
import torch

# Configuration
# Do when app start running??
max_seq_length = 2048
dtype = None
load_in_4bit = True

# Load the Fine-Tuned Model
model_path = "typhoon_fake_review_detector"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_path,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# -------------------------------------------

In [ ]:


FastLanguageModel.for_inference(model)

# should be the same System Prompt format as our training data
system_prompt = "You are Typhoon, a model that analyzes product reviews and outputs the likelihood of being fake."
test_review = "kuay"

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": f"""
    PRODUCT_TITLE: \"Targus PAUK10U Ultra Mini USB Keypad, Black\",
    REVIEW_TITLE: \"useful\" ,
    Review: \"{test_review}\"
    """}
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must be True for generation
    return_tensors = "pt",
).to("cuda")

# Generate Response
outputs = model.generate(
    input_ids = inputs,
    max_new_tokens = 64, # Adjust-able
    use_cache = True,
    temperature = 0.1,
)

# Decode and Print
# ทำยังไงให้มันไม่ print สะเปะสะปะ วะ
decoded_output = tokenizer.batch_decode(outputs)
print("\n--- Model Prediction ---")
print(decoded_output[0])


--- Model Prediction ---
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are Typhoon, a model that analyzes product reviews and outputs the likelihood of being fake.<|eot_id|><|start_header_id|>user<|end_header_id|>

PRODUCT_TITLE: "Targus PAUK10U Ultra Mini USB Keypad, Black",
    REVIEW_TITLE: "useful",
    Review: "kuay"<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{"fake_probability": 0.8}<|eot_id|>
